## Example code to test saved crystal structure data

In [1]:
import numpy as np
import pickle
import h5py
from tqdm import tqdm
from onsager import crystal, supercell

## Load the hdf5 file with the crystal structure and supercell data

In [2]:
with h5py.File("CrystData.h5", "r") as fl:
    lattice = np.array(fl["Lattice_basis_vectors"])
    superlatt = np.array(fl["SuperLatt"])
    siteIndtoR = np.array(fl["SiteIndToR"])
    RtoSiteInd = np.array(fl["RToSiteInd"])
    dxList = np.array(fl["dxList_1nn"])
    NNList = np.array(fl["NNsiteList_sitewise"])
    jumpNewIndices = np.array(fl["JumpSiteIndexPermutation"])
    GroupOpLatticeCartRotMatrices = np.array(fl["GroupOpLatticeCartRotMatrices"])
    GpermNNIdx = np.array(fl["GroupNNPermutation"])


jList = NNList[1:, 0]

crys = crystal.Crystal(lattice=lattice, basis=[[np.array([0., 0., 0.])]], chemistry=["A"])
superCell = supercell.ClusterSupercell(crys, superlatt)

In [3]:
dxList # contains the jump vectors.

array([[ 0. , -0.5,  0.5],
       [-0. ,  0.5, -0.5],
       [ 0.5,  0. , -0.5],
       [-0.5, -0. ,  0.5],
       [ 0. , -0.5, -0.5],
       [-0. ,  0.5,  0.5],
       [-0.5, -0.5,  0. ],
       [ 0.5,  0.5, -0. ],
       [ 0.5,  0. ,  0.5],
       [-0.5, -0. , -0.5],
       [ 0.5, -0.5,  0. ],
       [-0.5,  0.5, -0. ]])

## Verify Nearest neighbor sites under saved order of jumps

In [4]:
# Convert nearest neighbors to lattice vectors.
dxNNVecs = np.zeros_like(dxList).astype(int)
for jmp in range(dxList.shape[0]):
    dx = dxList[jmp]
    dxNN, _ = superCell.crys.cart2pos(dx)
    dxNNVecs[jmp, :] = dxNN[:]

In [5]:
for siteInd in tqdm(range(NNList.shape[1]), position=0, leave=True, ncols=65):
    assert NNList[0, siteInd] == siteInd
    ciSite, Rsite = superCell.ciR(siteInd)
    for jmp in range(NNList.shape[0] - 1):
        RsiteNew = Rsite + dxNNVecs[jmp]
        siteNew, _ = superCell.index(RsiteNew, (ciSite))
        assert NNList[jmp + 1, siteInd] == siteNew
    
        if siteInd == 0:
            assert jList[jmp] == siteNew

100%|████████████████████████| 512/512 [00:00<00:00, 3920.40it/s]


## Now test the group permutation of nearest neigbhors

In [6]:
# We first reconstruct the group operations dictionary
# This will let us test the consistency in the sequence of the group operations
considered = set()
GIndtoGDict = {}
for g in list(superCell.crys.G):
    cartrot = g.cartrot
    for rotInd in range(GroupOpLatticeCartRotMatrices.shape[0]):
        if np.allclose(cartrot, GroupOpLatticeCartRotMatrices[rotInd]):
            assert g not in considered # A group operation cannot be repeated 
            considered.add(g)
            GIndtoGDict[rotInd] = g

In [7]:
# Then test that the nearest neighbors have been consistently stored
for gInd, g in tqdm(GIndtoGDict.items(), position=0, leave=True, ncols=65):
    for jmp in range(dxList.shape[0]):
        jmpvec = dxList[jmp]
        jmpvecRot = superCell.crys.g_cart(g, jmpvec)
        
        idxnew = None
        count = 0
        for jmpNew in range(dxList.shape[0]):
            if np.allclose(dxList[jmpNew], jmpvecRot):
                count += 1
                idxnew = jmpNew
        assert count == 1
        assert GpermNNIdx[gInd, idxnew + 1] == jmp + 1

assert len(GIndtoGDict) == 48

100%|████████████████████████████| 48/48 [00:00<00:00, 97.03it/s]


## Now check re-indexing of sites after jumps, when sites are translated back under PBC so that the vacancy comes back at (0,0,0)

In [8]:
# Now for each jump, displace periodically and check
# Now test the nearest neighbor indexing
randomState = np.random.randint(1, 5, NNList.shape[1], dtype=np.int8)
print(randomState.shape)
# Put a vacancy at 0
randomState[0] = 0


for jmp in tqdm(range(dxList.shape[0]), position=0, leave=True, ncols=65):
    
    dxRVec, _ = superCell.crys.cart2pos(dxList[jmp])
    
    state2 = randomState.copy()
    assert state2[0] == 0 # check initially there was vacancy
    state2[0] = state2[jList[jmp]]
    state2[jList[jmp]] = 0
    state2Trans = np.zeros_like(state2)
    for site in range(NNList.shape[1]):
        ciSite, Rsite = superCell.ciR(site)
        RsiteNew = Rsite - dxRVec # translate by negative of vac jump
        siteIndNew, _ = superCell.index(RsiteNew, ciSite)
        state2Trans[siteIndNew] = state2[site]
    
    assert state2Trans[0] == 0
    assert np.array_equal(randomState[jumpNewIndices[jmp]], state2Trans)

(512,)


100%|████████████████████████████| 12/12 [00:00<00:00, 86.15it/s]


## Check the site index arrays

In [9]:
for siteInd in range(siteIndtoR.shape[0]):
    R = siteIndtoR[siteInd]
    siteIndCalc = R[0] * 8 * 8 + R[1] * 8 + R[2]
    assert siteIndCalc == siteInd
    assert RtoSiteInd[R[0], R[1], R[2]] == siteInd
    siteIndSuperCell , _ = superCell.index(R, (0,0))
    assert siteIndCalc == siteIndSuperCell

In [10]:
jList

array([ 15,  57, 449,  71, 448,  64,   7,   1,   8,  56, 456, 120])